In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
# imports
import pandas as pd
from collections import defaultdict
from mjol.base import *
from mjol.gan import *
from mjol.tools import *
import pyfastx
import os

In [2]:
def check_integrity(x) -> bool:
    for aid in x.lookup:
        for uid in x.lookup[aid]:
            if uid not in x.features:
                return False
    return True

In [30]:
o_fn = "/ccb/salz4-3/hji20/chm13/chm13v2.0_RefSeq_Liftoff_v5.2.gff3"
o_chm13 = GAn(
    file_name = o_fn,
    file_fmt = 'gff'
)
o_chm13.build_db()

In [31]:
n_genes = 0
for f in o_chm13.features.values():
    if f.feature_type == 'gene':
        n_genes += 1
check_integrity(o_chm13)

True

In [32]:
nw_fn = "/ccb/salz4-3/hji20/chm13/T2T_annotation_v5.3/output/mapped_reformatted.gff"
nw_chm13 = GAn(
    file_name = nw_fn,
    file_fmt = 'gff'
)
nw_chm13.build_db()

In [10]:
swap_fn = "/ccb/salz4-3/hji20/chm13/T2T_annotation_v5.3/intermediates/overlaps_8.csv"
swaps_df = pd.read_csv(swap_fn, index_col=0)

In [11]:
syn_df = swaps_df[swaps_df['synonym'] == True]
non_syn_df = swaps_df[swaps_df['synonym'] == False]

In [33]:
# variables for logging
added_lns = []
removed_lns = []
actions = dict()

In [13]:
def update_attributes(
    starter,
    nw_att,
    update_rule,
    verbose : bool = False
) -> dict:
    for o_k, nw_k in update_rule.items():
        if nw_k not in nw_att:
            if verbose:
                print(f'WARNING : expected {nw_k} in nw_att arg, but not found')
            continue
        starter[o_k] = nw_att[nw_k]
    return starter

In [34]:
cn_tbl = defaultdict(int)
for f in o_chm13.features.values():
    if f.feature_type == "gene":
        if 'gene' in f.attributes:
            gname = f.attributes['gene']
        elif 'gene_name' in f.attributes:
            gname = f.attributes['gene_name']
        cn_tbl[gname] += 1

In [15]:
nw_pfn = "/ccb/salz4-3/hji20/chm13/mapped_protein.fa"
nw_pfa = pyfastx.Fasta(nw_pfn)

In [35]:
vorf_tbl = dict()
for f in nw_chm13.features.values():
    if f.feature_type == "mRNA":
        if 'valid_ORF' not in f.attributes:
            vorf_tbl[f.aid] = 'None'
            continue
        if f.attributes['valid_ORF'] == 'True':
            pseq = nw_pfa[f.aid].seq
            # consider adding stop codon check?
            if '.' in pseq or pseq[0] != 'M':
                vorf_tbl[f.aid] = False
            else:
                vorf_tbl[f.aid] = True
        else:
            vorf_tbl[f.aid] = False

In [17]:
sorted_syn_df = syn_df.sort_values(by='gene_add') # must operate on sorted syn_df
sorted_non_syn_df = non_syn_df.sort_values(by='gene_add')

In [36]:
# NOTE: these changes are irreversible

btype_tbl = defaultdict(int)

curr_cn_tbl = defaultdict(int)
cn_ctr = 0
prev_add_gname = None # prev added gene name
txes_to_add = [] # tx to add manually
genes_to_check = [] # gene to check copy number attributes
resolved_ranges = dict()

for i, row in sorted_syn_df.iterrows():
    add_btype = row['biotype_add']
    del_btype = row['biotype_remove']
    btype_tbl[del_btype] += 1
    assert add_btype == "protein_coding" # sanity check

    o_gname = row['gene_remove']
    o_gid = row['ID_remove']
    o_uid = o_chm13.get_uid(o_gid)
    og = o_chm13.get_feature(o_uid)

    if cn_tbl[o_gname] > 1:
        genes_to_check.append(o_gname)
    
    nw_gname = row['gene_add']
    nw_gid = row['ID_add']
    nw_uid = nw_chm13.get_uid(nw_gid)
    ng = nw_chm13.get_feature(nw_uid)

    # update cn_ctr
    if prev_add_gname != nw_gname:
        curr_cn_tbl[prev_add_gname] = cn_ctr
        prev_add_gname = nw_gname
        cn_ctr = 0
    else:
        cn_ctr += 1
    
    if del_btype == "protein_coding":
        removed_lns.append(og.to_gff_entry(include_children=True))
        
        if cn_ctr > 0:
            new_gaid = f'{ng.aid.split('_')[0]}_{cn_ctr}'
        else:
            new_gaid = ng.aid.split('_')[0]

        update_rule = {
            'gene_name' : 'Name',
            'db_xref' : 'Dbxref',
            'description' : 'description',
            'gene' : 'gene',
            'gene_biotype' : 'gene_biotype',
            'coverage' : 'coverage',
            'sequence_ID' : 'sequence_ID',
        }

        g_att = update_attributes(
            starter = { 'ID' : new_gaid },
            nw_att = ng.attributes,
            update_rule = update_rule
        )

        # additional attribute updates
        g_att['valid_ORFs'] = og.attributes['valid_ORFs']
        g_att['extra_copy_number'] = str(cn_ctr)
        g_att['copy_num_ID'] = f'{g_att['ID']}_{cn_ctr}' if cn_ctr == 0 else g_att['ID']
        
        # update gene
        og.attributes = g_att
        del o_chm13.lookup[og.aid]
        og.aid = g_att['ID']
        o_chm13.lookup[og.aid] = [og.uid]

        tgt_tx = ng.children[0]
        tgt_itrn_chain = tgt_tx.get_chain('exon')
        add_tgt = True

        for tx in og.children:
            if tgt_itrn_chain == tx.get_chain('exon'):
                add_tgt = False
            
            temp = tx.attributes
            if cn_ctr > 0:
                new_taid = f'{temp['ID']}_{cn_ctr}'
            else:
                new_taid = temp['ID']
            
            t_att = update_attributes(
                starter = {
                    'ID' : new_taid
                },
                nw_att = g_att,
                update_rule = {
                    'Parent' : 'ID',
                    'db_xref' : 'db_xref',
                    'gene' : 'gene',
                    'extra_copy_number' : 'extra_copy_number'
                }
            )
            
            t_att = update_attributes(
                starter = t_att,
                nw_att = temp,
                update_rule = {
                    'tag' : 'tag',
                    'transcript_biotype' : 'transcript_biotype',
                    'matches_ref_protein' : 'matches_ref_protein',
                    'valid_ORF' : 'valid_ORF'
                }
            )

            # update tx
            tx.attributes = t_att
            del o_chm13.lookup[tx.aid]
            tx.aid = new_taid
            tx.paid = og.aid
            o_chm13.lookup[tx.aid] = [tx.uid]

            for child in tx.children:
                temp2 = child.attributes
                
                c_att = update_attributes(
                    starter = {},
                    nw_att = t_att,
                    update_rule = {
                        'Parent' : 'ID',
                        'db_xref' : 'db_xref',
                        'gene' : 'gene',
                        'tag' : 'tag',
                        'transcript_biotype' : 'transcript_biotype',
                        'extra_copy_number' : 'extra_copy_number'
                    }
                )

                # update child
                child.attributes = c_att
                assert not child.aid # sanity check
                child.paid = tx.aid
        added_lns.append(og.to_gff_entry(include_children=True))
        resolved_ranges[og.aid] = (og.chr, og.start, og.end)
        actions[i] = f'RENAME;{del_btype};{o_gid};{add_btype};{og.aid}'
    else:
        removed_lns.append(og.to_gff_entry(include_children=True))
        if cn_ctr > 0:
            new_gaid = f'{ng.aid.split('_')[0]}_{cn_ctr}'
        else:
            new_gaid = ng.aid.split('_')[0]
        
        update_rule = {
            'gene_name' : 'Name',
            'db_xref' : 'Dbxref',
            'description' : 'description',
            'gene' : 'gene',
            'gene_biotype' : 'gene_bioype',
            'coverage' : 'coverage',
            'sequence_ID' : 'sequence_ID'
        }
        
        g_att = update_attributes(
            starter = { 'ID' : new_gaid },
            nw_att = ng.attributes,
            update_rule = update_rule
        )

        g_att['extra_copy_number'] = str(cn_ctr)
        g_att['copy_num_ID'] = f'{g_att['ID']}_{cn_ctr}' if cn_ctr == 0 else g_att['ID']
        
        # add gene
        ng.attributes = g_att
        ng.aid = new_gaid
        o_chm13.add_feature(
            feature = ng,
            include_children = False
        )

        for tx in ng.children:
            temp = tx.aid.split('_')
            if len(temp) == 2:
                base = tx.aid
            else:
                base = '_'.join(temp[:-1])
            new_taid = f'{base}_{cn_ctr}' if cn_ctr > 0 else base

            tx_att = update_attributes(
                starter = {
                    'ID' : new_taid,
                    'Parent' : new_gaid,
                    'transcript_biotype' : 'mRNA',
                    'extra_copy_number' : str(cn_ctr)
                },
                nw_att = tx.attributes,
                update_rule = {
                    "db_xref" : "Dbxref",
                    "gene" : "gene",
                    "tag" : "tag",
                    "matches_ref_protein" : "matches_ref_protein",
                    'valid_ORF' : 'valid_ORF'
                }
            )

            # add tx
            tx.attributes = tx_att
            tx.paid = new_gaid
            tx.aid = new_taid
            tx.feature_type = "transcript"
            o_chm13.add_feature(
                feature = tx,
                include_children = False
            )

            for child in tx.children:
                c_att = update_attributes(
                    starter = {},
                    nw_att = tx_att,
                    update_rule = {
                        'Parent' : 'ID',
                        'db_xref' : 'db_xref',
                        'gene' : 'gene',
                        'tag' : 'tag',
                        'transcript_biotype' : 'transcript_biotype',
                        'extra_copy_number' : 'extra_copy_number'
                    }
                )

                # add child
                child.attributes = c_att
                child.aid = None
                child.paid = tx.aid
                o_chm13.add_feature(
                    feature = child,
                    include_children = False
                )
        
        # delete og and its descendants
        o_chm13.pop_feature(og.uid, include_children=True)

        resolved_ranges[ng.aid] = (ng.chr, ng.start, ng.end)
        added_lns.append(ng.to_gff_entry(include_children=True))
        actions[i] = f'RENAME;{del_btype};{o_gid};{add_btype};{ng.aid}'

In [37]:
btype_tbl

defaultdict(int,
            {'protein_coding': 138,
             'pseudogene': 15,
             'lncRNA': 1,
             'transcribed_pseudogene': 1})

In [38]:
n_genes2 = 0
for f in o_chm13.features.values():
    if f.feature_type == 'gene':
        n_genes2 += 1
check_integrity(o_chm13)

assert n_genes == n_genes2

In [39]:
rslved_by_chr = defaultdict(list)

for gid in resolved_ranges:
    chr, st, en = resolved_ranges[gid]
    rslved_by_chr[chr].append((gid, st, en))

In [23]:
def is_overlapping(x_st, x_en, y_st, y_en) -> bool:
    """
    x - query
    y - reference
    """
    return max(x_st, y_st) <= min(x_en, y_en)

In [ ]:
# MANUAL, IGNORE, ADD, REMOVE, AS_IS logics

manual_df = None
added_tbl = defaultdict(list)
removed_tbl = defaultdict(list)

for _, sub_df in sorted_non_syn_df.groupby('gene_add'):
    if len(sub_df) > 1:
        if manual_df is None:
            manual_df = sub_df
        else:
            manual_df = pd.concat([manual_df, sub_df], axis=0)
        for i, row in sub_df.iterrows():
            actions[i] = 'MANUAL'
    else:
        for i, row in sub_df.iterrows():
            is_ovlp = False # skip legal overlaps, applying logics of transitivity described above
            for r in rslved_by_chr[row['chromosome']]:
                gid, st, en = r
                if is_overlapping(
                    x_st = int(row['start_remove']),
                    x_en = int(row['end_remove']),
                    y_st = int(st),
                    y_en = int(en)
                ):
                    is_ovlp = True
                    break
            if is_ovlp:
                actions[i] = f'IGNORE;{gid}'
                break
    
            nw_gid = row['ID_add']
            nw_gname = row['gene_add']
            nw_uid = nw_chm13.get_uid(nw_gid)
            ng = nw_chm13.get_feature(nw_uid)

            if row['overlapping_in_MANE']:
                if str(curr_cn_tbl[nw_gname]) != ng.attributes['extra_copy_number']:
                    print(f'CRITICAL : update copy number for {nw_gid}')
                ng.children[0].feature_type = 'transcript'
                out = o_chm13.add_feature(
                    feature = ng,
                    include_children = True
                )
                added_lns.append(out)
                added_tbl[nw_gname].append(nw_gid)
                removed_lns.append('None\n')
                actions[i] = f'ADD;{ng.aid}'
            else:
                nt = ng.children[0]
                if nt.aid not in vorf_tbl:
                    raise KeyError()
                if not vorf_tbl[nt.aid]:
                    actions[i] = f'AS_IS;INVALID;{ng.aid}'
                    continue
                o_gname = row['gene_remove']
                if cn_tbl[o_gname] > 1:
                    genes_to_check.append(o_gname)
                o_gid = row['ID_remove']

                if not row['gene_remove_in_MANE']:
                    if o_gid in o_chm13.lookup:
                        out = o_chm13.pop_feature(
                            uid = o_chm13.get_uid(o_gid),
                            include_children = True
                        )
                        removed_lns.append(out)
                        removed_tbl[o_gname].append(o_gid)
                    else:
                        removed_lns.append('None\n')
                    
                    nt.feature_type = 'transcript'
                    out = o_chm13.add_feature(
                        feature = ng,
                        include_children = True
                    )
                    added_lns.append(out)
                    added_tbl[nw_gname].append(nw_gid)
                    actions[i] = f'REMOVE;{row['biotype_remove']};{o_gid};ADD;{row['biotype_add']};{ng.aid}'
                else:
                    og = o_chm13.get_feature(o_chm13.get_uid(o_gid))
                    winning_condition = False
                    if og.attributes['valid_ORFs'] == '0':
                        winning_condition = True
                        print(f'CRITICAL : replace {og.aid} (broken) with {ng.aid} (intact)')
                    # satisfy a strictly longer condition?
                    if row['gene_add_protein_length'] > row['gene_remove_protein_length']:
                        winning_condition = False
                        print(f'CRITICAL : replace {og.aid} (shorter) with {ng.aid} (longer)')
                    actions[i] = f'AS_IS;WIN={winning_condition}'
                    

In [41]:
# append actions to swaps_df
assert len(actions) == len(swaps_df) # sanity check
sorted_actions = dict(sorted(actions.items()))
swaps_df['action'] = list(sorted_actions.values())

In [42]:
print(txes_to_add) # no MANE tx to manually add

[]


In [43]:
# write out results

assert len(added_lns) == len(removed_lns)

out_dir = "/ccb/salz4-3/hji20/chm13/out"
swaps_df.to_csv(os.path.join(out_dir, 'edit_infos.csv'), index=False)

with open(os.path.join(out_dir, 'edited_entries.txt'), 'w') as fh:
    for i in range(len(added_lns)):
        fh.write(f'>>>\n{removed_lns[i]}<<<\n{added_lns[i]}')

o_chm13.to_gff3(os.path.join(out_dir, 'out.gff'))

In [ ]:
manual_df.to_csv(os.path.join(out_dir, 'manual_edit_infos.csv'), index=False)

with open(os.path.join(out_dir, 'genes_to_check.txt'), 'w') as fh:
    fh.write('\n'.join(genes_to_check))

with open(os.path.join(out_dir, 'added_genes.csv'), 'w') as fh:
    for gname in added_tbl:
        for gid in added_tbl[gname]:
            fh.write(f'{gname},{gid}\n')

with open(os.path.join(out_dir, 'removed_genes.csv'), 'w') as fh:
    for gname in removed_tbl:
        for gid in removed_tbl[gname]:
            fh.write(f'{gname},{gid}\n')

### manual curation

In [80]:
for _, sub_df in manual_df.groupby('gene_add'):
    actions = []
    for i, row in sub_df.iterrows():
        action = 'None'
        chr = row['chromosome']
        st = row['start_remove']
        en = row['end_remove']
        for r in rslved_by_chr[chr]:
            gid, st2, en2 = r
            if is_overlapping(
                x_st = st,
                x_en = en,
                y_st = st2,
                y_en = en2
            ):
                action = 'IGNORE'
                break
        if action == 'None':
            if row['overlapping_in_MANE']:
                action = 'ADD'
            else:
                nw_gid = row['ID_add']
                nw_uid = nw_chm13.get_uid(nw_gid)
                ng = nw_chm13.get_feature(nw_uid)
                nt = ng.children[0]
                if not vorf_tbl[nt.aid]:
                    action = f'AS_IS;INVALID;{ng.aid}'
                else:
                    if not row['gene_remove_in_MANE']:
                        action = f'REMOVE;{row['ID_remove']};{row['ID_add']}'
                    else:
                        action = f'COMPARE'
        actions.append(action)
    
    print(f'{set(sub_df['gene_add'])}\t{set(actions)}')

{'ARB2A'}	{'IGNORE'}
{'CNTNAP3'}	{'REMOVE;LOC100420438;CNTNAP3', 'REMOVE;CNTNAP3C;CNTNAP3', 'REMOVE;LOC101930090;CNTNAP3'}
{'EIF3CL'}	{'COMPARE', 'REMOVE;MIR6862-2;EIF3CL'}
{'ENTREP2'}	{'IGNORE'}
{'FAM246B'}	{'REMOVE;DGCR5;FAM246B', 'REMOVE;FAM246C;FAM246B'}
{'FAM90A12'}	{'COMPARE', 'REMOVE;FAM90A24P_3;FAM90A12_7', 'REMOVE;FAM90A24P_4;FAM90A12_6', 'REMOVE;FAM90A24P_2;FAM90A12_8'}
{'FAM90A13'}	{'REMOVE;FAM90A15P;FAM90A13_2', 'REMOVE;FAM90A5P;FAM90A13_1', 'COMPARE'}
{'FAM90A24'}	{'COMPARE'}
{'GK3'}	{'AS_IS;INVALID;GK3_2', 'AS_IS;INVALID;GK3_1'}
{'GOLGA8Q'}	{'REMOVE;LOC100288482_1;GOLGA8Q', 'REMOVE;RN7SL196P;GOLGA8Q', 'COMPARE'}
{'IGBP1C'}	{'IGNORE'}
{'KCNE1'}	{'REMOVE;LOC124900475;KCNE1', 'REMOVE;LOC105379505;KCNE1'}
{'LOC128125814'}	{'REMOVE;MIR616;LOC128125814', 'ADD'}
{'MSANTD7'}	{'REMOVE;LOC100421372;MSANTD7', 'ADD'}
{'OR4F21'}	{'AS_IS;INVALID;OR4F21_2', 'AS_IS;INVALID;OR4F21', 'AS_IS;INVALID;OR4F21_1', 'AS_IS;INVALID;OR4F21_3'}
{'POTEB3'}	{'COMPARE', 'REMOVE;RNU6-631P;POTEB3'}
{'POT